In [7]:
import os
import cv2
import glob
import numpy as np

In [2]:
input_dir = "/home/chuongh/vm2m/data/syn/benchmark/real"
out_dir = "/home/chuongh/vm2m/data/syn/benchmark/real_mosaic"

In [5]:
all_video_names = sorted(os.listdir(os.path.join(input_dir, "fgr")))

In [58]:
def load_video_paths(video_name):
    # Load fgr
    all_image_paths = glob.glob(os.path.join(input_dir, "fgr", video_name) + "/*.jpg")
    all_image_paths.sort()

    # Load pha
    all_pha_paths = glob.glob(os.path.join(input_dir, "pha", video_name) + "/*/*.png")
    all_pha_paths.sort()

    # Load xmem
    all_mask_paths = glob.glob(os.path.join(input_dir, "xmem", video_name) + "/*/*.png")
    all_mask_paths.sort()

    return all_image_paths, all_pha_paths, all_mask_paths

def compose_frame_row(all_image_paths, all_pha_paths, all_mask_paths):
    max_h = 0
    
    all_images = []
    all_phas = []
    all_masks = []

    # Load image, mask, pha
    for image_path, pha_path, mask_path in zip(all_image_paths, all_pha_paths, all_mask_paths):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        max_h = max(max_h, image.shape[0])
        pha = cv2.imread(pha_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        all_images.append(image)
        all_phas.append(pha)
        all_masks.append(mask)
    
    # Resize
    for i in range(len(all_images)):
        all_images[i] = cv2.resize(all_images[i], (all_images[i].shape[1] * max_h // all_images[i].shape[0], max_h))
        all_phas[i] = cv2.resize(all_phas[i], (all_phas[i].shape[1] * max_h // all_phas[i].shape[0], max_h), cv2.INTER_NEAREST)
        all_masks[i] = cv2.resize(all_masks[i], (all_masks[i].shape[1] * max_h // all_masks[i].shape[0], max_h), cv2.INTER_NEAREST)
    
    # Compose images
    all_images = np.concatenate(all_images, axis=1)

    # Padding mask, alpha and stack
    final_mask = np.zeros((*all_images.shape[:2], len(all_masks)), dtype=np.uint8)
    final_pha = np.zeros_like(final_mask)
    x = 0
    for i in range(len(all_masks)):
        w = all_masks[i].shape[1]
        final_mask[:, x:x+ w, i] = all_masks[i]
        final_pha[:, x:x+ w, i] = all_phas[i]
        x += w
    return all_images, final_pha, final_mask

def compose_video_row(all_image_paths, all_pha_paths, all_mask_paths):
    all_images = []
    all_phas = []
    all_masks = []
    for i in range(len(all_image_paths[0])):
        image_paths = [x[i] for x in all_image_paths]
        pha_paths = [x[i] for x in all_pha_paths]
        mask_paths = [x[i] for x in all_mask_paths]
        images, phas, masks = compose_frame_row(image_paths, pha_paths, mask_paths)
        if i == 0:
            all_images = [images]
            all_phas = [phas]
            all_masks = [masks]
        else:
            all_images.append(images)
            all_phas.append(phas)
            all_masks.append(masks)
    return all_images, all_phas, all_masks

def gen_video(idx, video_names):
    random = np.random.RandomState(idx)
    n_videos = len(video_names)
    # video_names = random.choice(all_video_names, n_videos, replace=False)

    min_n_frames = 100000
    all_video_infos = []
    for video_name in video_names:
        all_image_paths, all_pha_paths, all_mask_paths = load_video_paths(video_name)
        min_n_frames = min(min_n_frames, len(all_image_paths))
        all_video_infos.append([all_image_paths, all_pha_paths, all_mask_paths])

    # Select n_frames
    for frame_info in all_video_infos:
        frame_ids = random.choice(range(len(frame_info[0])), min_n_frames, replace=False)
        frame_info[0] = [frame_info[0][i] for i in frame_ids]
        frame_info[0].sort()
        frame_info[1] = [frame_info[1][i] for i in frame_ids]
        frame_info[1].sort()
        frame_info[2] = [frame_info[2][i] for i in frame_ids]
        frame_info[2].sort()
    if n_videos < 4:
        all_image_paths = [x[0] for x in all_video_infos]
        all_pha_paths = [x[1] for x in all_video_infos]
        all_mask_paths = [x[2] for x in all_video_infos]
        images, phas, masks = compose_video_row(all_image_paths, all_pha_paths, all_mask_paths)
    else:
        all_image_paths = [x[0] for x in all_video_infos[:2]]
        all_pha_paths = [x[1] for x in all_video_infos[:2]]
        all_mask_paths = [x[2] for x in all_video_infos[:2]]
        img_row0, pha_row0, mask_row0 = compose_video_row(all_image_paths, all_pha_paths, all_mask_paths)
        all_image_paths = [x[0] for x in all_video_infos[2:]]
        all_pha_paths = [x[1] for x in all_video_infos[2:]]
        all_mask_paths = [x[2] for x in all_video_infos[2:]]
        img_row1, pha_row1, mask_row1 = compose_video_row(all_image_paths, all_pha_paths, all_mask_paths)
        width = max(img_row0[0].shape[1], img_row1[0].shape[1])
        images = []
        for r1, r2 in zip(img_row0, img_row1):
            r1 = cv2.resize(r1, (width, int(r1.shape[0] * width // r1.shape[1])))
            r2 = cv2.resize(r2, (width, int(r2.shape[0] * width // r2.shape[1])))
            images.append(np.concatenate([r1, r2], axis=0))
        phas = []
        masks = []
        for i in range(len(pha_row0)):
            r1 = pha_row0[i]
            r2 = pha_row1[i]
            new_h1 = int(r1.shape[0] * width // r1.shape[1])
            new_h2 = int(r2.shape[0] * width // r2.shape[1])
            r1 = cv2.resize(r1, (width, new_h1))
            r2 = cv2.resize(r2, (width, new_h2))
            pha = np.zeros((new_h1 + new_h2, width, r1.shape[2] + r2.shape[2]), dtype=np.uint8)
            mask = np.zeros_like(pha)
            pha[:new_h1, :, :r1.shape[2]] = r1
            pha[new_h1:, :, r1.shape[2]:] = r2
            phas.append(pha)
            r1 = mask_row0[i]
            r2 = mask_row1[i]
            r1 = cv2.resize(r1, (width, new_h1))
            r2 = cv2.resize(r2, (width, new_h2))
            mask[:new_h1, :, :r1.shape[2]] = r1
            mask[new_h1:, :, r1.shape[2]:] = r2
            masks.append(mask)

    # Save video
    video_name = format(idx, "02d")
    # Save image, mask, pha
    for i in range(len(images)):
        out_path = os.path.join(out_dir, "fgr", video_name, format(i, "03d") + ".jpg")
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        cv2.imwrite(out_path, images[i])

        for j in range(masks[i].shape[2]):
            out_path = os.path.join(out_dir, "xmem", video_name, format(i, "03d"), format(j, "03d") + ".png")
            os.makedirs(os.path.dirname(out_path), exist_ok=True)
            cv2.imwrite(out_path, masks[i][:, :, j])
            out_path = os.path.join(out_dir, "pha", video_name, format(i, "03d"), format(j, "03d") + ".png")
            os.makedirs(os.path.dirname(out_path), exist_ok=True)
            cv2.imwrite(out_path, phas[i][:, :, j])

In [55]:

random = np.random.RandomState(0)
video_names2 = all_video_names + random.choice(all_video_names, 2, replace=False).tolist()
video_names2 = random.permutation(video_names2).tolist()

# Check no 2 consecutive videos are the same
for i in range(len(video_names2) - 1):
    assert video_names2[i] != video_names2[i + 1]

video_names3 = all_video_names + random.choice(all_video_names, 3 * 10 - len(all_video_names), replace=False).tolist()
is_done = False
while not is_done:
    print("Permuting 3")
    video_names3 = random.permutation(video_names3).tolist()
    is_done = True
    for i in range(len(video_names3) - 2):
        if i %3 == 0 and len(set(video_names3[i:i+3])) < 3:
            is_done = False
            break

video_names4 = all_video_names * 2 + random.choice(all_video_names, 4 * 10 - len(all_video_names) * 2, replace=True).tolist()
is_done = False
while not is_done:
    print("Permuting 4")
    video_names4 = random.permutation(video_names4).tolist()
    is_done = True
    for i in range(len(video_names4) - 3):
        if i % 4 == 0 and len(set(video_names4[i:i+4])) < 4:
            is_done = False
            break

Permuting 3
Permuting 3
Permuting 3
Permuting 3
Permuting 4
Permuting 4
Permuting 4
Permuting 4
Permuting 4
Permuting 4
Permuting 4


In [62]:
i = 0
for i in range(10):
    print("process 2", i)
    gen_video(i, video_names2[i*2: i*2 + 2])
    i += 1
for i in range(10):
    print("process 3", i)
    gen_video(i + 10, video_names3[i*3: i*3 + 3])
    i += 1
for i in range(10):
    print("process 4", i)
    gen_video(i + 20, video_names4[i*4: i*4 + 4])
    i += 1

process 2 0


process 2 1
process 2 2
process 2 3
process 2 4
process 2 5
process 2 6
process 2 7
process 2 8
process 2 9
process 3 0
process 3 1
process 3 2
process 3 3
process 3 4
process 3 5
process 3 6
process 3 7
process 3 8
process 3 9
process 4 0
process 4 1
process 4 2
process 4 3
process 4 4
process 4 5
process 4 6
process 4 7
process 4 8
process 4 9


Bad pipe message: %s [b'\x15O\x15\x14\xd1\x12\x92;\xe0t\x8c\xac\xc3\xf9uE\xeaY BB|)\xd52"&\xadW\x8a\xbe\xfc\xa1\xdd\xc3\xd9\xe0[+R\xc7\xa25?\x1c\xfb^\x00f\x8c\x18\x00>\x13\x02\x13\x03\x13\x01\xc0,\xc00\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0+\xc0/\x00\x9e\xc0$\xc0(\x00k\xc0#\xc0\'\x00g\xc0\n\xc0\x14\x009\xc0\t\xc0\x13\x003\x00\x9d\x00\x9c\x00=\x00<\x005\x00/\x00\xff\x01\x00\x01u\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x03\x03\x02\x03\x03\x01\x02\x01\x03\x02\x02\x02\x04\x02\x05\x02\x06\x02\x00+\x00\t\x08\x03\x04\x03\x03\x03\x02\x03\x01\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00']
Bad pipe message: %s [b' ^\xe2\xd5U\xae\x82\xc5j"[\x11\x11\x1e\x05\xa3^P\xda\x1e\xcc\x0f\x0f-\xb9NZ\x1b']


In [61]:
video_names2

['Pexels_Videos_2795385',
 'production_ID_5158552',
 'production_ID_4594921',
 'pexels-artem-podrez-5457692',
 'production_ID_5197613',
 'production_ID_4736798',
 'production_ID_4625474',
 '3048929',
 'pexels-rodnae-productions-5642515',
 'production_ID_4065480',
 'production_ID_4728157',
 'production_ID_4918184',
 'production_ID_4531303',
 'production_ID_4098957',
 'production_ID_4099014',
 'production_ID_4265392',
 'production_ID_4825047',
 'production_ID_4170843',
 'production_ID_4099014',
 'Pexels_Videos_2795385']

In [42]:
for i in range(30):
    print(i)
    if i < 10:
        gen_video(i, 2)
    elif i < 20:
        gen_video(i, 3)
    else:
        gen_video(i, 4)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [28]:
all_video_infos

[[['/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00021.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00041.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00061.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00101.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00111.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00121.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00211.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00221.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00271.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00291.jpg',
   '/home/chuongh/vm2m/data/syn/benchmark/real/fgr/production_ID_4594921/frame00331.jpg',
   '/home/